# Execute embedding and search on remote server

In [1]:
!pip install paramiko

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 76.6 MB/s eta 0:00:00


Transfer data file

In [ ]:
import getpass
import subprocess
import os
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display

# Your code to be executed when the button is clicked
def send_data_to_server():
# Load environment variables from .env file
    load_dotenv()
    
    # Define a variable based on a value from .env
    remote_host = os.getenv("server")
    filename = text_widget.value
    remote_path = "/home/codechallenge/InnoClass/backend/classEngine/data/"
    print(filename)
    #password = getpass.getpass(f"Enter password for {remote_host}: ")
    #command = ["scp", "-o", f"StrictHostKeyChecking=no", "-o", f"PreferredAuthentications=password", filename, f"{remote_host}:{remote_path}"]
    command = ["scp", filename, f"{remote_host}:{remote_path}"]
    try:
        process = subprocess.Popen(command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        stdout, stderr = process.communicate() # Append newline to password
        return_code = process.returncode
    
        if return_code == 0:
            print("SCP command executed successfully.")
            print("Output:", stdout)
        else:
            print(f"SCP command failed with error (code {return_code}):")
            print("Error:", stderr)
    
    except FileNotFoundError:
        print("Error: scp command not found.")
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    print("Sending data to server...")
    # Example: Accessing a text widget value
    if hasattr(text_widget, 'value'):
        print(f"Text widget value: {text_widget.value}")
    # Add your server communication logic here

# Create the button
send_button = widgets.Button(description="Send to Server")

# Create an optional text widget for example
text_widget = widgets.Text(description="Data filename:")

# Define the button's click event handler
def on_button_click(b):
    send_data_to_server()

# Attach the event handler to the button's click event
send_button.on_click(on_button_click)

# Display the widgets
display(text_widget)
display(send_button)


Configuration file for embedding and search

In [ ]:
conifg={
    "filename_sdg": "",
    "filename_texts"]:"",
    'Prompt':"",
    'extracted_text':"",
    "index":"hybrid_search_index",
    "model:'all-MiniLM-L6-v2'
} 

In [ ]:
    # 2. Define KNN search part
knn_query = {
        "field": "embedding",
        "query_vector": query_embedding.tolist(),
        "k": k,
        "num_candidates": num_candidates # Higher value increases accuracy but uses more resources
}

    # 3. Define Keyword search part (BM25)
keyword_query = {
        "match": {
            "cleaned_text": {
                "query": cleaned_query
                # Add options like "fuzziness": "AUTO" if needed
            }
        }
}

search_body = {
        "queries": [
            {"knn": knn_query},
            {"query": keyword_query}
        ],
        "rank": {
            "rrf": {
                "window_size": num_candidates + k,
                "rank_constant": 20
            }
        },
        "size": k,
        "_source": ["original_data", "cleaned_text"]
}

In [23]:
import paramiko
import os
from dotenv import load_dotenv
load_dotenv()


def execute_remote_command(hostname, username, private_key_path, command):
    """Executes a command on a remote machine using SSH keys."""
    try:
        private_key = paramiko.Ed25519Key.from_private_key_file(private_key_path)
        client = paramiko.SSHClient()
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        client.connect(hostname, username=username, pkey=private_key)

        stdin, stdout, stderr = client.exec_command(command)
        output = stdout.read().decode()
        error = stderr.read().decode()

        client.close()

        if error:
            print(f"Error: {error}")
        return output

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

hostname = os.getenv("server")
username = os.getenv("username")
private_key_path = os.getenv("key")
command = "cd /home/codechallenge/InnoClass/backend/; docker-compose -f docker-compose.python.yml up fastapi-app"  # Replace with your desired command
result = execute_remote_command(hostname,username, private_key_path,command)

if result:
    print("Command output:")
    print(result)

Error: elasticsearch is up-to-date
Starting fastapi-app ... 
Starting fastapi-app ... done

Command output:
Attaching to fastapi-app
fastapi-app      | Running python command
fastapi-app      | /usr/local/lib/python3.9/site-packages/elasticsearch/_sync/client/__init__.py:403: SecurityWarning: Connecting to 'https://elasticsearch:9200' using TLS with verify_certs=False is insecure
fastapi-app      |   _transport = transport_class(
fastapi-app      | /usr/local/lib/python3.9/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elasticsearch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
fastapi-app      |   warnings.warn(
fastapi-app      | /usr/local/lib/python3.9/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elasticsearch'. Adding certificate verification is strongl